In [41]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [42]:
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [43]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/anoop-vs/nlp-climate-change/refs/heads/main/main_data.csv"
)

In [44]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package stopwords to /Users/nafis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/nafis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/nafis/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [45]:
def preprocess(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    words = text.split()
    stopword_list = set(stopwords.words("english"))
    words = [w for w in words if w not in stopword_list]
    stemmer = PorterStemmer()
    words = [stemmer.stem(w) for w in words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w) for w in words]
    text = " ".join(words)
    return text

In [46]:
df["content"] = df["content"].apply(preprocess)

In [47]:
from imblearn.over_sampling import RandomOverSampler

In [48]:
oversampler = RandomOverSampler(
    sampling_strategy={"Negative": 1842, "Positive": 1822, "Neutral": 1842}
)

In [49]:
X = df.drop("label", axis=1)
y = df["label"]

In [50]:
X_resampled, y_resampled = oversampler.fit_resample(X, y)

In [51]:
df_resampled = pd.concat([X_resampled, y_resampled], axis=1)

In [52]:
X = df_resampled["content"]

In [53]:
y = df_resampled["label"]

In [54]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

COUNTVECTORIZER - LOGISTIC REGRESSION


In [55]:
# Convert the text data to numerical vectors using count Vectorizer
vectorizer = CountVectorizer()

X_train_cv = vectorizer.fit_transform(X_train)
X_test_cv = vectorizer.transform(X_test)

In [56]:
# Initialize a logistic regression model with default parameters
lr_model = LogisticRegression(C=0.1, solver = 'lbfgs')

# Fit the model on the training data
lr_model.fit(X_train_cv, y_train)

# Predict the target variable on the testing data
y_pred = lr_model.predict(X_test_cv)

# Calculate the accuracy score of the model on the testing data
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy score
print('Accuracy:', accuracy)

Accuracy: 0.7903811252268602


### COUNTVECTORIZER - RANDOM FOREST


In [57]:
clf = RandomForestClassifier()
clf.fit(X_train_cv, y_train)
y_pred = clf.predict(X_test_cv)
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.93      0.97      0.95       389
     Neutral       0.89      0.85      0.87       370
    Positive       0.87      0.87      0.87       343

    accuracy                           0.90      1102
   macro avg       0.90      0.90      0.90      1102
weighted avg       0.90      0.90      0.90      1102



### **COUNTVECTORIZER - SVM**


In [58]:
classifier = SVC(C=10, gamma=10, kernel="linear")
classifier.fit(X_train_cv, y_train)
y_pred = classifier.predict(X_test_cv)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8784029038112523


In [59]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.89      0.98      0.93       389
     Neutral       0.88      0.82      0.85       370
    Positive       0.86      0.83      0.84       343

    accuracy                           0.88      1102
   macro avg       0.88      0.88      0.88      1102
weighted avg       0.88      0.88      0.88      1102



### COUNTVECTORIZER - DECISION TREE


In [60]:
dt_model = DecisionTreeClassifier(
    max_depth=None, max_features="auto", min_samples_leaf=1, min_samples_split=2
)
dt_model.fit(X_train_cv, y_train)
y_pred = dt_model.predict(X_test_cv)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7976406533575318


/Users/nafis/code/python/climate change sentiment analysis/.venv311/lib/python3.11/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


In [61]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.89      0.94      0.92       389
     Neutral       0.74      0.74      0.74       370
    Positive       0.75      0.70      0.72       343

    accuracy                           0.80      1102
   macro avg       0.79      0.79      0.79      1102
weighted avg       0.79      0.80      0.80      1102



### COUNTVECTORIZER - NAIVE BAYES


In [62]:
clf = MultinomialNB()
clf.fit(X_train_cv, y_train)
y_pred = clf.predict(X_test_cv)

In [63]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.77      0.95      0.85       389
     Neutral       0.88      0.66      0.75       370
    Positive       0.79      0.78      0.78       343

    accuracy                           0.80      1102
   macro avg       0.81      0.80      0.80      1102
weighted avg       0.81      0.80      0.80      1102

